## Import Libraries

In [3]:
import os
import sys

## Read The PDF

In [4]:
data='data.pdf'

In [5]:
data

'data.pdf'

## Convert into chunk

In [1]:
#import the library
from langchain.text_splitter import RecursiveCharacterTextSplitter